#### Microeconometrics HW 5
06-20-2021, by Imryoung Jeong

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas import DataFrame

###### Question 1. 
---
<code> Y = D*Y(1) + (1-D)*Y(0) --> Y = delta * D + 0.5 + U </code>
###### Question 2. 
---
<i> No. Defier means : if Z == 0, D(0) == 1 and if Z == 1, D(1) == 0. To satisfy these conditions, by given definitions of D, -0.5 >= V and V > 1.5, which is contradiction. Hence, in this case, one can eliminate the possibility of being 'defier'. </i> 

In [24]:
mean = [0,0]
cov = [[1,0.3],[0.3,1]]

u,v = np.random.multivariate_normal(mean, cov, 20000).T
z = np.random.binomial(1, 0.3, 20000).T

df = pd.DataFrame({'u':u, 'v':v, 'z':z})
df.head(3)

,u,v,z
0,-2.039379,-1.857972,0
1,-0.474685,-0.486780,1
2,-1.682439,0.188908,1


###### Question 3. Generate U,V, Y(1), Y(0), D(1), D(0)

In [31]:
df['Y(1)'] = 0.5 + 1 + df['u']
df['Y(0)'] = 0.5 + df['u']

df['D'] = (-0.5 + 2*df['z'] > df['v']).astype(int)

df['D(1)'] = ( -0.5 + 2 > df['v']).astype(int)
df['D(0)'] = ( -0.5 > df['v']).astype(int)

df.head(5)

,u,v,z,Y(1),Y(0),D,D(1),D(0)
0,-2.039379,-1.857972,0,-0.539379,-1.539379,1,1,1
1,-0.474685,-0.486780,1,1.025315,0.025315,1,1,0
2,-1.682439,0.188908,1,-0.182439,-1.182439,1,1,0
3,0.019762,1.265857,0,1.519762,0.519762,0,1,0
4,0.245757,0.367440,1,1.745757,0.745757,1,1,0


###### Question 4. Generate Always-taker, Complier, Never-taker, and Defier indicator

In [36]:
df['Always-taker'] = ( (df['D(1)'] == 1) & (df['D(0)'] ==  1) ).astype(int)
df['Complier'] = ( (df['D(1)'] == 1) & (df['D(0)'] ==  0) ).astype(int)
df['Never-taker'] = ( (df['D(1)'] == 0) & (df['D(0)'] ==  0) ).astype(int)
df['Defier'] = ( (df['D(1)'] == 0) & (df['D(0)'] ==  1) ).astype(int)

df.head(5)

,u,v,z,Y(1),Y(0),D,D(1),D(0),Always-taker,Complier,Never-taker,Defier
0,-2.039379,-1.857972,0,-0.539379,-1.539379,1,1,1,1,0,0,0
1,-0.474685,-0.486780,1,1.025315,0.025315,1,1,0,0,1,0,0
2,-1.682439,0.188908,1,-0.182439,-1.182439,1,1,0,0,1,0,0
3,0.019762,1.265857,0,1.519762,0.519762,0,1,0,0,1,0,0
4,0.245757,0.367440,1,1.745757,0.745757,1,1,0,0,1,0,0


###### Qustion 5. ATE

In [52]:
df['t'] = df['Y(1)']-df['Y(0)']
print ("Average Treatment Effect : " + str(df['t'].mean()))

Average Treatment Effect : 1.0


###### Question 6. ATE for subgroups

In [55]:
df['category'] = np.where(df['Always-taker'] == 1, 'Always-taker',
                         (np.where(df['Complier'] == 1, 'Complier', 'Never-taker')))


df.groupby('category')['t'].mean()

category
Always-taker    1.0
Complier        1.0
Never-taker     1.0
Name: t, dtype: float64

###### Question 7. OLS estimator

In [56]:
import statsmodels.formula.api as sm

In [57]:
df['Y'] = df['D']*df['Y(1)'] + (1-df['D'])*df['Y(0)']

result = sm.ols(formula = 'Y ~ D', data = df).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.103
Method:                 Least Squares   F-statistic:                     2303.
Date:                Sun, 20 Jun 2021   Prob (F-statistic):               0.00
Time:                        17:04:46   Log-Likelihood:                -28018.
No. Observations:               20000   AIC:                         5.604e+04
Df Residuals:                   19998   BIC:                         5.606e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6616      0.010     67.086      0.0

###### Question 8. IV estimator

In [63]:
print("IV estimator : " + str(df.z.cov(df.Y) / df.z.cov(df.D)))

IV estimator : 0.9646120280347426


###### Question 9. Heterogeneous DGP 

In [72]:
df2 = df[['u','v','z']].copy()

df2['delta'] = pd.Series(np.random.normal(1, 1, 20000).T)

df2['Y(1)'] = 0.5 + df2['delta'] + df2['u']
df2['Y(0)'] = 0.5 + df2['u']

df2['D(1)'] = ( -0.5 + 2 > df2['v']).astype(int)
df2['D(0)'] = ( -0.5 > df2['v']).astype(int)

df2['D'] = (-0.5 + 2*df2['z'] > df2['v']).astype(int)
df2['Y'] = df2['D']*df2['Y(1)'] + (1-df2['D'])*df2['Y(0)']

df2['Always-taker'] = ( (df2['D(1)'] == 1) & (df2['D(0)'] ==  1) ).astype(int)
df2['Complier'] = ( (df2['D(1)'] == 1) & (df2['D(0)'] ==  0) ).astype(int)
df2['Never-taker'] = ( (df2['D(1)'] == 0) & (df2['D(0)'] ==  0) ).astype(int)
df2['Defier'] = ( (df2['D(1)'] == 0) & (df2['D(0)'] ==  1) ).astype(int)

df2['category'] = np.where(df2['Always-taker'] == 1, 'Always-taker',
                         (np.where(df2['Complier'] == 1, 'Complier', 'Never-taker')))

df2['t'] = df2['Y(1)']-df2['Y(0)']

df2.head(4)

,u,v,z,delta,Y(1),Y(0),D(1),D(0),D,Y,Always-taker,Complier,Never-taker,Defier,category,t
0,-2.039379,-1.857972,0,0.083774,-1.455605,-1.539379,1,1,1,-1.455605,1,0,0,0,Always-taker,0.083774
1,-0.474685,-0.486780,1,1.136291,1.161606,0.025315,1,0,1,1.161606,0,1,0,0,Complier,1.136291
2,-1.682439,0.188908,1,3.025714,1.843276,-1.182439,1,0,1,1.843276,0,1,0,0,Complier,3.025714
3,0.019762,1.265857,0,1.988488,2.508250,0.519762,1,0,0,0.519762,0,1,0,0,Complier,1.988488


In [73]:
print ("(5) Average Treatment Effect : " + str(df2['t'].mean()))

(5) Average Treatment Effect : 0.9979091270105508


In [74]:
df2.groupby('category')['t'].mean()

category
Always-taker    1.007891
Complier        0.994188
Never-taker     0.985411
Name: t, dtype: float64

<i> ATE and LATE are similar? </i>

In [76]:
result2 = sm.ols(formula = 'Y ~ D', data = df2).fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     1556.
Date:                Sun, 20 Jun 2021   Prob (F-statistic):               0.00
Time:                        17:34:14   Log-Likelihood:                -32120.
No. Observations:               20000   AIC:                         6.424e+04
Df Residuals:                   19998   BIC:                         6.426e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6616      0.012     54.644      0.0

In [77]:
print("(8) IV estimator : " + str(df2.z.cov(df2.Y) / df2.z.cov(df2.D)))

(8) IV estimator : 0.9563901480893489


<i> Although 'delta' varies across observations, it doesn't depend on V or X (not essentially heterogeneous). In this case, IV estimator identifies LATE, but LATE is not so much different from ATE. (but OLS estimator seems to have a bias?)</i>

###### Question 10. Essential Heterogeneity
---
<i>As 'delta' here depends on v and treatment effect D is also affected by v, 'delta' and D are not independent (thus yielding essential heterogeneity)</i>

In [90]:
df3 = df[['u','v','z']].copy()

delta = []
for i in range(len(df)) :
    
    mean = df3['v'][i] +1
    delta.append(np.random.normal(mean, 1, 1)[0])
    
df3['delta'] = delta

df3['Y(1)'] = 0.5 + df3['delta'] + df3['u']
df3['Y(0)'] = 0.5 + df3['u']

df3['D(1)'] = ( -0.5 + 2 > df3['v']).astype(int)
df3['D(0)'] = ( -0.5 > df3['v']).astype(int)

df3['D'] = (-0.5 + 2*df3['z'] > df3['v']).astype(int)
df3['Y'] = df3['D']*df3['Y(1)'] + (1-df3['D'])*df3['Y(0)']

df3['Always-taker'] = ( (df3['D(1)'] == 1) & (df3['D(0)'] ==  1) ).astype(int)
df3['Complier'] = ( (df3['D(1)'] == 1) & (df3['D(0)'] ==  0) ).astype(int)
df3['Never-taker'] = ( (df3['D(1)'] == 0) & (df3['D(0)'] ==  0) ).astype(int)
df3['Defier'] = ( (df3['D(1)'] == 0) & (df3['D(0)'] ==  1) ).astype(int)

df3['category'] = np.where(df3['Always-taker'] == 1, 'Always-taker',
                         (np.where(df3['Complier'] == 1, 'Complier', 'Never-taker')))

df3['t'] = df3['Y(1)']-df3['Y(0)']

df3.head(4)

,u,v,z,delta,Y(1),Y(0),D(1),D(0),D,Y,Always-taker,Complier,Never-taker,Defier,category,t
0,-2.039379,-1.857972,0,0.571741,-0.967638,-1.539379,1,1,1,-0.967638,1,0,0,0,Always-taker,0.571741
1,-0.474685,-0.486780,1,0.309967,0.335282,0.025315,1,0,1,0.335282,0,1,0,0,Complier,0.309967
2,-1.682439,0.188908,1,0.759308,-0.423131,-1.182439,1,0,1,-0.423131,0,1,0,0,Complier,0.759308
3,0.019762,1.265857,0,3.307007,3.826768,0.519762,1,0,0,0.519762,0,1,0,0,Complier,3.307007


In [91]:
print ("(5) Average Treatment Effect : " + str(df3['t'].mean()))

(5) Average Treatment Effect : 0.9985637643736837


In [92]:
df3.groupby('category')['t'].mean()

category
Always-taker   -0.120424
Complier        1.363185
Never-taker     2.939881
Name: t, dtype: float64

<i> ATE conditional on Complier (=LATE) is most similar to ATE, but two are not necessarily the same. </i>

In [94]:
result3 = sm.ols(formula = 'Y ~ D', data = df3).fit()
print(result3.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     29.74
Date:                Sun, 20 Jun 2021   Prob (F-statistic):           5.00e-08
Time:                        19:44:09   Log-Likelihood:                -35661.
No. Observations:               20000   AIC:                         7.133e+04
Df Residuals:                   19998   BIC:                         7.134e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6616      0.014     45.779      0.0

In [95]:
print("(8) IV estimator : " + str(df3.z.cov(df3.Y) / df3.z.cov(df3.D)))

(8) IV estimator : 1.3328744879070766


<i> IV estimator (=1.33) is almost identical with the LATE (=1.36), which is consistent with what we've learned.</i>